In [24]:
#Import library
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

from multiprocessing import *
import warnings
warnings.filterwarnings("ignore")

import time
import gc


from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier

# Regularized Greedy Forest
from rgf.sklearn import RGFClassifier     # https://github.com/fukatani/rgf_python
from catboost import CatBoostClassifier

In [31]:
#### Load Data
train = pd.read_csv('/Users/siero5335/Desktop/Safe Driver Prediction/train.csv',na_values=-1)
test = pd.read_csv('/Users/siero5335/Desktop/Safe Driver Prediction/test.csv',na_values=-1)
id_test = test['id'].values

In [12]:
ps_cal = train.columns[train.columns.str.startswith('ps_calc')] 
train = train.drop(ps_cal,axis =1)
test = test.drop(ps_cal,axis=1)
train.shape

(595212, 39)

In [5]:
def missing_value(df):
    col = df.columns
    for i in col:
        if df[i].isnull().sum()>0:
            df[i].fillna(df[i].mode()[0],inplace=True)

In [6]:
missing_value(train)
missing_value(test)

In [7]:
def uniq(df):
    col = df.columns
    for i in col:
        print('\n Unique value of "{}" is "{}" '.format(i,df[i].nunique()))
        #print(df[i].unique())
uniq(train)


 Unique value of "id" is "595212" 

 Unique value of "target" is "2" 

 Unique value of "ps_ind_01" is "8" 

 Unique value of "ps_ind_02_cat" is "4" 

 Unique value of "ps_ind_03" is "12" 

 Unique value of "ps_ind_04_cat" is "2" 

 Unique value of "ps_ind_05_cat" is "7" 

 Unique value of "ps_ind_06_bin" is "2" 

 Unique value of "ps_ind_07_bin" is "2" 

 Unique value of "ps_ind_08_bin" is "2" 

 Unique value of "ps_ind_09_bin" is "2" 

 Unique value of "ps_ind_10_bin" is "2" 

 Unique value of "ps_ind_11_bin" is "2" 

 Unique value of "ps_ind_12_bin" is "2" 

 Unique value of "ps_ind_13_bin" is "2" 

 Unique value of "ps_ind_14" is "5" 

 Unique value of "ps_ind_15" is "14" 

 Unique value of "ps_ind_16_bin" is "2" 

 Unique value of "ps_ind_17_bin" is "2" 

 Unique value of "ps_ind_18_bin" is "2" 

 Unique value of "ps_reg_01" is "10" 

 Unique value of "ps_reg_02" is "19" 

 Unique value of "ps_reg_03" is "5012" 

 Unique value of "ps_car_01_cat" is "12" 

 Unique value of "ps_car

In [8]:
def category_type(df):
    col = df.columns
    for i in col:
        if df[i].nunique()<=104:
            df[i] = df[i].astype('category')
category_type(train)
category_type(test)

In [9]:
cat_col = [col for col in train.columns if '_cat' in col]
print(cat_col)

['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat']


In [10]:
bin_col = [col for col in train.columns if 'bin' in col]
print(bin_col)

['ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin', 'ps_calc_20_bin']


In [13]:
tot_cat_col = list(train.select_dtypes(include=['category']).columns)

other_cat_col = [c for c in tot_cat_col if c not in cat_col+ bin_col]
other_cat_col

['target',
 'ps_ind_01',
 'ps_ind_03',
 'ps_ind_14',
 'ps_ind_15',
 'ps_reg_01',
 'ps_reg_02',
 'ps_car_11',
 'ps_car_15']

In [14]:
num_col = [c for c in train.columns if c not in tot_cat_col]
num_col.append('ps_car_15')
train['ps_car_15'] = train['ps_car_15'].astype('float32')
test['ps_car_15'] = test['ps_car_15'].astype('float32')
num_col.remove('id')
num_col

['ps_reg_03', 'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15']

In [15]:
#Final categorical columns to be consider
tot_cat_col.remove('ps_car_15')
tot_cat_col.remove('target')
train['ps_car_15'] = train['ps_car_15'].astype('float32')

In [16]:
def outlier(df,columns):
    for i in columns:
        quartile_1,quartile_3 = np.percentile(df[i],[25,75])
        quartile_f,quartile_l = np.percentile(df[i],[1,99])
        IQR = quartile_3-quartile_1
        lower_bound = quartile_1 - (1.5*IQR)
        upper_bound = quartile_3 + (1.5*IQR)
        print(lower_bound,upper_bound)
        print(quartile_f,quartile_l)
        
        df[i].loc[df[i] < lower_bound] = quartile_f
        df[i].loc[df[i] > upper_bound] = quartile_l
        
outlier(train,num_col)

0.08484029175 1.54909582495
0.4183300133 1.8521946442
0.190569415 0.525658351
0.3155946768 0.5656854249
0.31788087655 1.25917611615
0.448300509774 1.61761689551
0.2891566485 0.4608812941
0.2887905816 0.5234500931
1.66274082661 4.77123749256
0.0 3.7416574955


In [17]:
def OHE(df1,df2,column):
    cat_col = column
    #cat_col = df.select_dtypes(include =['category']).columns
    len_df1 = df1.shape[0]
    
    df = pd.concat([df1,df2],ignore_index=True)
    c2,c3 = [],{}
    
    print('Categorical feature',len(column))
    for c in cat_col:
        if df[c].nunique()>2 :
            c2.append(c)
            c3[c] = 'ohe_'+c
    
    df = pd.get_dummies(df, prefix=c3, columns=c2,drop_first=True)

    df1 = df.loc[:len_df1-1]
    df2 = df.loc[len_df1:]
    print(df1.shape)
    print(df2.shape)
    return df1,df2

In [18]:
train1,test1 = OHE(train,test,tot_cat_col)

Categorical feature 32
(595212, 244)
(892816, 244)


In [19]:
train = train1.drop(['target','id'],axis=1)
target_train = train1['target'].astype('category')
test = test1.drop(['target','id'],axis=1)
del train1,test1

In [20]:
class Ensemble(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models

    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)

        folds = list(StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=2017).split(X, y))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):

            S_test_i = np.zeros((T.shape[0], self.n_splits))

            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
                y_holdout = y[test_idx]

                print ("Fit %s fold %d" % (str(clf).split('(')[0], j+1))
                clf.fit(X_train, y_train)
                cross_score = cross_val_score(clf, X_train, y_train, cv=self.n_splits, scoring='roc_auc')
                print("    cross_score: %.5f" % (cross_score.mean()))
                y_pred = clf.predict_proba(X_holdout)[:,1]                

                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict_proba(T)[:,1]
            S_test[:, i] = S_test_i.mean(axis=1)

            print("     Model score: %.5f\n" % roc_auc_score(y, S_train[:,i]))

        self.base_preds = S_test
        
        # Log odds transformation
        almost_zero = 1e-12
        almost_one = 1 - almost_zero  # To avoid division by zero
        S_train[S_train>almost_one] = almost_one
        S_train[S_train<almost_zero] = almost_zero
        S_train = np.log(S_train/(1-S_train))
        S_test[S_test>almost_one] = almost_one
        S_test[S_test<almost_zero] = almost_zero
        S_test = np.log(S_test/(1-S_test))
        
        results = cross_val_score(self.stacker, S_train, y, cv=self.n_splits, scoring='roc_auc')
        print("Stacker score: %.5f" % (results.mean()))

        self.stacker.fit(S_train, y)
        print( 'Coefficients:', self.stacker.coef_ )

        res = self.stacker.predict_proba(S_test)[:,1]
        return res

In [21]:
# LightGBM params
lgb_params = {}
lgb_params['learning_rate'] = 0.02
lgb_params['n_estimators'] = 650
lgb_params['max_bin'] = 10
lgb_params['subsample'] = 0.8
lgb_params['subsample_freq'] = 10
lgb_params['colsample_bytree'] = 0.8   
lgb_params['min_child_samples'] = 500
lgb_params['random_state'] = 99


lgb_params2 = {}
lgb_params2['n_estimators'] = 1090
lgb_params2['learning_rate'] = 0.02
lgb_params2['colsample_bytree'] = 0.3   
lgb_params2['subsample'] = 0.7
lgb_params2['subsample_freq'] = 2
lgb_params2['num_leaves'] = 16
lgb_params2['random_state'] = 99


lgb_params3 = {}
lgb_params3['n_estimators'] = 1100
lgb_params3['max_depth'] = 4
lgb_params3['learning_rate'] = 0.02
lgb_params3['random_state'] = 99

lgb_params4 = {}
lgb_params4['learning_rate'] = 0.02
lgb_params4['n_estimators'] = 650
lgb_params4['max_bin'] = 10
lgb_params4['subsample'] = 0.8
lgb_params4['subsample_freq'] = 10
lgb_params4['colsample_bytree'] = 0.8   
lgb_params4['min_child_samples'] = 500
lgb_params4['random_state'] = 71
lgb_params['reg_lambda'] = 1.3

lgb_params5 = {}
lgb_params5['n_estimators'] = 1090
lgb_params5['learning_rate'] = 0.02
lgb_params5['colsample_bytree'] = 0.3   
lgb_params5['subsample'] = 0.7
lgb_params5['subsample_freq'] = 2
lgb_params5['num_leaves'] = 16
lgb_params5['random_state'] = 71
lgb_params['reg_lambda'] = 1.3

lgb_params6 = {}
lgb_params6['n_estimators'] = 1100
lgb_params6['max_depth'] = 4
lgb_params6['learning_rate'] = 0.02
lgb_params6['random_state'] = 71
lgb_params['reg_lambda'] = 1.3
lgb_params['subsample'] = 0.8

# XGBoost params
xgb_params1 = {}
xgb_params1['objective'] = 'binary:logistic'
xgb_params1['learning_rate'] = 0.02
xgb_params1['n_estimators'] = 1100
xgb_params1['max_depth'] = 4
xgb_params1['subsample'] = 0.8
xgb_params1['colsample_bytree'] = 0.8 
xgb_params1['min_child_weight'] = 2.4073000000000002
xgb_params1['reg_alpha'] = 8.0701999999999998
xgb_params1['seed'] = 71
xgb_params1['gamma'] = 0.15110000000000001
xgb_params1['reg_lambda'] =  2.0125999999999999
xgb_params1['scale_pos_weight'] =  2.2281

xgb_params2 = {}
xgb_params2['objective'] = 'binary:logistic'
xgb_params2['learning_rate'] = 0.02
xgb_params2['n_estimators'] = 1100
xgb_params2['max_depth'] = 4
xgb_params2['subsample'] = 0.8
xgb_params2['colsample_bytree'] = 0.8 
xgb_params2['min_child_weight'] = 2.4073000000000002
xgb_params2['reg_alpha'] = 8.0701999999999998
xgb_params2['seed'] = 99
xgb_params2['gamma'] = 0.15110000000000001
xgb_params2['reg_lambda'] = 2.0125999999999999
xgb_params2['scale_pos_weight'] =  2.2281

xgb_params3 = {}
xgb_params3['objective'] = 'binary:logistic'
xgb_params3['learning_rate'] = 0.02
xgb_params3['n_estimators'] = 1100
xgb_params3['max_depth'] = 4
xgb_params3['subsample'] = 0.8
xgb_params3['colsample_bytree'] = 0.8 
xgb_params3['min_child_weight'] = 2.4073000000000002
xgb_params3['reg_alpha'] = 8.0701999999999998
xgb_params3['seed'] = 114514
xgb_params3['gamma'] = 0.15110000000000001
xgb_params3['reg_lambda'] = 2.0125999999999999
xgb_params3['scale_pos_weight'] =  2.2281

# Regularized Greedy Forest params
rgf_params = {}
rgf_params['max_leaf'] = 2000
rgf_params['learning_rate'] = 0.2
rgf_params['algorithm'] = "RGF_Sib"
rgf_params['test_interval'] = 100
rgf_params['min_samples_leaf'] = 3 
rgf_params['reg_depth'] = 1.0
rgf_params['l2'] = 0.5  
rgf_params['sl2'] = 0.005

#CatBoost params
cat_params = {}
cat_params['iterations'] = 900
cat_params['depth'] = 4
cat_params['rsm'] = 0.95
cat_params['learning_rate'] = 0.03
cat_params['l2_leaf_reg'] = 3.5  
cat_params['border_count'] = 8
cat_params['gradient_iterations'] = 4

cat_params2 = {}
cat_params2['iterations'] = 900
cat_params2['depth'] = 8
cat_params2['rsm'] = 0.95
cat_params2['learning_rate'] = 0.03
cat_params2['l2_leaf_reg'] = 3.5  
cat_params2['border_count'] = 8
cat_params2['gradient_iterations'] = 4

cat_params3 = {}
cat_params3['iterations'] = 900
cat_params3['depth'] = 10
cat_params3['rsm'] = 0.95
cat_params3['learning_rate'] = 0.03
cat_params3['l2_leaf_reg'] = 3.5  
cat_params3['border_count'] = 8
cat_params3['gradient_iterations'] = 4

In [27]:
cat_model = CatBoostClassifier(**cat_params)

cat_model2 = CatBoostClassifier(**cat_params2)

cat_model3 = CatBoostClassifier(**cat_params3)

lgb_model = LGBMClassifier(**lgb_params)

lgb_model2 = LGBMClassifier(**lgb_params2)

lgb_model3 = LGBMClassifier(**lgb_params3)

lgb_model4 = LGBMClassifier(**lgb_params4)

lgb_model5 = LGBMClassifier(**lgb_params5)

lgb_model6 = LGBMClassifier(**lgb_params6)

xgb_model1 = XGBClassifier(**xgb_params1)

xgb_model2 = XGBClassifier(**xgb_params2)

xgb_model3 = XGBClassifier(**xgb_params3)

rgf_model = RGFClassifier(**rgf_params)

log_model = LogisticRegression(fit_intercept=False)

ridge_model = Ridge(fit_intercept=False)

In [29]:
log_model = LogisticRegression(fit_intercept=False)

stack = Ensemble(n_splits=3,
        stacker = log_model,
        base_models = (cat_model, lgb_model, lgb_model2, lgb_model3))        
        
y_pred = stack.fit_predict(train, target_train, test)        

Fit <catboost.core.CatBoostClassifier object at 0x1ae477160> fold 1
    cross_score: 0.63335
Fit <catboost.core.CatBoostClassifier object at 0x1ae477160> fold 2
    cross_score: 0.63336
Fit <catboost.core.CatBoostClassifier object at 0x1ae477160> fold 3
    cross_score: 0.63217
     Model score: 0.63458

Fit LGBMClassifier fold 1
    cross_score: 0.63351
Fit LGBMClassifier fold 2
    cross_score: 0.63388
Fit LGBMClassifier fold 3
    cross_score: 0.63388
     Model score: 0.63674

Fit LGBMClassifier fold 1
    cross_score: 0.63336
Fit LGBMClassifier fold 2
    cross_score: 0.63456
Fit LGBMClassifier fold 3
    cross_score: 0.63462
     Model score: 0.63707

Fit LGBMClassifier fold 1
    cross_score: 0.63225
Fit LGBMClassifier fold 2
    cross_score: 0.63171
Fit LGBMClassifier fold 3
    cross_score: 0.63031
     Model score: 0.63445

Stacker score: 0.63779
Coefficients: [[ 0.12554282  0.41200915  0.50906663 -0.04661392]]


In [32]:
sub = pd.DataFrame()
sub['id'] = id_test
sub['target'] = y_pred
sub.to_csv('stacked_13_lgb_cat.csv', index=False)